## ResNet18 코드 실습
- https://www.youtube.com/watch?v=671BsKl8d0E

#### ResNet18 모델 정의 및 인스턴스 초기화

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os

In [ ]:
# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module): # 논문에서 그림으로 나오는 layers에서 하나의 Block
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절 -> dimension을 줄임)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False), # 동일한 차원으로 변경을 함
                nn.BatchNorm2d(planes)
            )

    def forward(self, x): # forward 함수는 호출하지 않아도 model 객체를 데이터와 함께 호출하면 자동으로 실행됨
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10): # 본 모델은 capacity가 커서 num_classess를 1000으로 바꾸고 ImageNet을 학습시켜도 됨
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False) # 처음 input layer에 들어가는 채널의 크기가 1이다. MINST는 gray scale이기 때문
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) # num_blocks=2개 만큼 BasicBlock을 실행하고 BasicBlock에서는 2개의 layers가 있으므로 layer1에서는 총 4개의 conv layer을 지남
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) # layer2도 4개
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) # 3도 4개
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) # 4도 4개
        self.linear = nn.Linear(512, num_classes) # 총 16개 layers이고 마지막에 fully connected layer을 가지고 있음

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1) # 위에 layers를 만들때 처음 layer에만 stride 값이 2가 되게끔 함. 논문 그림을 보면 너비와 높이가 처음 BasicBlock을 들어갈때 반으로 줄어듦
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x): # forward 함수는 호출하지 않아도 model 객체를 데이터와 함께 호출하면 자동으로 실행됨
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out # 최종적으로 10개의 클래스가 나옴


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2]) # 각각 2번씩 BasicBlock을 사용함

#### 데이터셋 다운로드 및 불러오기

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])
# train용 dataset과 test용 dataset을 torchvision에 있는 데이터로 가지고옴
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid poten

#### 환경 설정 및 학습 함수 정의

In [ ]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.01
file_name = 'resnet18_mnist.pt' # 모델파일 저장용

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader): # train_loader을 통해서 매번 batch size 만큼 데이터를 뽑고 학습시킨다.
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs) # 데이터를 네트워크에 넣는다.
        loss = criterion(benign_outputs, targets)
        loss.backward() # 크로스 엔트로피를 이용해 backward를 한다.

        optimizer.step() # 전체 모델 파라미터를 업데이트 할 수 있도록 한다.
        train_loss += loss.item() # 학습중에 loss값 출력을 위해
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch): # train과 비슷하다.
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

# Learning rate를 바꿔가면서 진행
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 5: # 만약 epoch가 5를 넘는다면 lr을 10으로 나눔
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습 진행

In [ ]:
for epoch in range(0, 10):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Current batch: 0
Current benign train accuracy: 0.1015625
Current benign train loss: 2.362291097640991

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.15586525201797485

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02070758305490017

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08490730822086334

Current batch: 400
Current benign train accuracy: 0.9765625
Current benign train loss: 0.046947915107011795

Total benign train accuarcy: 96.03833333333333
Total benign train loss: 59.511239054612815

[ Test epoch: 0 ]

Test accuarcy: 98.74
Test average loss: 0.0003861032015760429
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.007969546131789684

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.020577574148774147

Current batch: 200
Current benign train accurac